In [1]:
import numpy as np
import pandas as pd
import pingouin as pg
import pickle
from itertools import product
import time
#from pathos.multiprocessing import ProcessingPool as Pool

#from multiprocessing import Pool

In [2]:
#taken without change from Peter's code
# Function to transform similarity matrices within each dictionary key to arrays
def transform_rdm(dictionary_data):
    arrays = {}

    # Create boolean matrix and only retain upper triangle. Uses first key, so assumes all keys have same size
    mask = np.ones(dictionary_data[list(dictionary_data.keys())[0]].shape, dtype='bool')
    #print(f"1{mask}")
    mask[np.tril_indices(len(dictionary_data[list(dictionary_data.keys())[0]]))] = False
    #print(f"2{mask}")
    for dict_idx, data_type in enumerate(list(dictionary_data.keys())):
        arrays[data_type] = dictionary_data[data_type].values[mask]  # Convert dataframe to array and mask
    return arrays

In [3]:
# Function to generate null distribution
def null_dist(paired_data, perms=500):
    #print('rng')
    rng = np.random.default_rng()
    #print('shuffle')
    shuffle_data = paired_data  # Copy to new dataframe as some functions may apply directly to original data
    #print('max')
    num_sub_comparisons = np.max(shuffle_data.index.values)  # Base comparisons off data as this varies (dropped na)
    #print('shuffled_corr')
    shuffled_corr = np.full(perms, np.nan)  # Empty array for null distribution

    for perm_idx in range(0, perms):
        #print(perm_idx)
        # Shuffle fMRI data
        sublist_a = rng.choice(num_sub_comparisons, size=num_sub_comparisons, replace=False)
        shuffle_data['fMRI'] = shuffle_data['fMRI'].iloc[sublist_a].reset_index(drop=True)

        """shuffled_corr[perm_idx] = pg.partial_corr(data=shuffle_data, x="fMRI", y="behavior",
                                                  covar=["age", "sex", "movement"], method='spearman')["r"][0]"""
        # Run correlation #He used pg.partial_corr, as I don't have covariate I will only use pg.corr
        shuffled_corr[perm_idx] = pg.corr(x=shuffle_data['fMRI'], y=shuffle_data['behavior'],
                                                  method='spearman')["r"][0] #x, y = names of column in data; covar = name of covariate in data ->remove it

    return shuffled_corr

In [4]:
# Function to generate null distribution for suspense ratings
def null_dist_pearson(x, y, perms=500): #perms = je ne sais pas trop quoi, à quoi correspond la valeur par défaut?
    rng = np.random.default_rng()
    shuffled_corr = np.full(perms, np.nan)  # Empty array for null distribution

    for perm_idx in range(0, perms):
        # Shuffle fMRI data
        sublist_a = rng.choice(len(x), size=len(x), replace=False) #generate len(x) number from 0 to len(x)
        x_new = np.array([x])[0][sublist_a]

        # Run correlation
        shuffled_corr[perm_idx] = pg.corr(x_new, y, method="pearson")["r"][0]

    return shuffled_corr

In [5]:
# Load  data
pkl_file = open("Pairwise_Data.pkl", "rb")
pairwise_data = pickle.load(pkl_file)
behavioral_arrays = transform_rdm(pairwise_data["behavior"])  # Transform behavioral/covariate data into arrays
fMRI_arrays = transform_rdm(pairwise_data["fMRI"])
pkl_file.close()


In [ ]:
#suspense rating thing. A adapter pour moi

"""suspense_ratings = np.load("avg_suspense.npy")
adjusted_suspense_ratings = np.block([np.repeat(np.mean(suspense_ratings[0:2]), 2), suspense_ratings[0:-2]])"""

In [6]:
# Initialize variables
subjects = len(pairwise_data['fMRI']['Left Amygdala_x_7Networks_LH_SalVentAttn_Med_1']) #take on brain region combination at random
num_perms = 10000 ###nbr of permutation to build the distribution
statistical_tests = pd.DataFrame(columns=["Behavior", "Connection", "r", "p", "perm_p"])
suspense_tests = {}

In [7]:
fMRI_arrays

{'Left Amygdala_x_7Networks_LH_Vis_1': array([-0.08424938975884064, -0.20444194116667247, 0.02634835849952273,
        -0.06453096650491, 0.39521565880139514, 0.007772826690991921,
        -0.15534603283763224, 0.12741776514112582, 0.09227623830332604,
        -0.19372019152811765], dtype=object),
 'Left Amygdala_x_7Networks_LH_Vis_2': array([0.025248713344547362, 0.0673253056304116, -0.021253843542144407,
        -0.12175785536379952, 0.39379008824054085, -0.0488410937191369,
        -0.22645389454878426, 0.06734332079031817, 0.020239377059791275,
        0.021752497449854646], dtype=object),
 'Left Amygdala_x_7Networks_LH_Vis_3': array([0.04740109603299829, -0.11577038515668825, 0.013577280952413255,
        -0.041324838831784866, 0.10350501379283646, 0.022421591180732576,
        0.01202441543361017, 0.01525624595299389, 0.13240560375012256,
        -0.23929648532996087], dtype=object),
 'Left Amygdala_x_7Networks_LH_Vis_4': array([0.03198285550576359, -0.11217774443164182, -0.03072

In [88]:
behavioral_arrays.keys()

dict_keys(['cov_total', 'DASS_dep', 'DASS_anx', 'DASS_str', 'bas_d', 'bas_f', 'bas_r', 'bis', 'BIG5_ext', 'BIG5_agr', 'BIG5_con', 'BIG5_neu', 'BIG5_ope', 'erq_cr', 'erq_es'])

In [177]:
connection_x_behavior2 = list(product(list(behavioral_arrays.keys())[:5], list(fMRI_arrays.keys())))
connection_x_behavior2[19508],connection_x_behavior2[20333] #fin de DASS_anx connection_x_behavior2[29262]

(('DASS_anx', 'Left Amygdala_x_7Networks_LH_Vis_1'),
 ('DASS_anx', 'Right Amygdala_x_Right Accumbens'))

In [178]:
from multiprocessing import Pool
connection_x_behavior2 = list(product(list(behavioral_arrays.keys())[:5], list(fMRI_arrays.keys())))
for idx, iteration in enumerate(connection_x_behavior2[19508:20333]):
    print(f"Iteration {idx} of {len(connection_x_behavior2)}")
    start_time = time.time()

    behavior = iteration[0]
    connection = iteration[1]

    behavioral_data = behavioral_arrays[behavior]

    # Format data
    pairwise_arrays = np.vstack((behavioral_data, fMRI_arrays[connection])).transpose() ###enlever age, sex, movement pour moi
    #print(pairwise_arrays)
    pairwise_dataframe = pd.DataFrame(data=pairwise_arrays, columns=['behavior', 'fMRI'], dtype='float32')###enlever age, sex, movement pour moi

    pairwise_dataframe = pairwise_dataframe.dropna().reset_index(drop=True)  # account for subjects with missing  ###usefull for me? 
    if not ('TR-' in connection):  # if movie-wide analysis, use absolute values for all covariates/behavior  
        pairwise_dataframe.loc[:, pairwise_dataframe.columns != 'fMRI'] = \
            np.absolute(pairwise_dataframe.loc[:, pairwise_dataframe.columns != 'fMRI'])  # all columns that aren't fMRI ###only behavior for me since I don't have 
    # Statistical test
    correlation = pg.corr(x=pairwise_dataframe['fMRI'], y=pairwise_dataframe['behavior'], method='spearman') ###gives n, ouliers, r, CI95, p-val, BF10, power

    # Null distribution
    """threads = 7
    with Pool(threads) as pool:
        n = pool.amap(null_dist, [pairwise_dataframe] * threads)
    pool.join()
    pool.close()
    null_distribution=n.get()"""
    print('Before pool')
    """with Pool (processes=7) as pool: #processes = number of thread (coeur) I use on my computer
        null_distribution = pool.map(null_dist, pairwise_dataframe)
    pool.close()"""
    print('After Pool')
    #null_distribution = np.block(null_distribution)
    null_distribution = null_dist(pairwise_dataframe)
    # Test Fisher transformed correlations. Absolute null z less then absolute observed z
    greater_less_sum = np.sum(np.absolute(np.arctanh(null_distribution)) <= np.absolute(np.arctanh(correlation.r[0])))  ###compte le nombre de fois que 
    ###la valeur que l'on veut tester est plus petite que la valeur significative
    permutation_significance = 1.0 - (greater_less_sum / num_perms)
    #permutation_significance = np.nan ###initialise à np.nan si la ligne précédente ne compte aucun résultat significatif (donc somme =0)

    statistical_tests.loc[len(statistical_tests)+1] = [behavior, connection, correlation.r[0], correlation['p-val'][0],
                                  permutation_significance]

    #print("--- %s seconds ---" % (time.time() - start_time))
statistical_tests

Iteration 0 of 48770
Before pool
After Pool
Iteration 1 of 48770
Before pool
After Pool
Iteration 2 of 48770
Before pool
After Pool
Iteration 3 of 48770
Before pool
After Pool
Iteration 4 of 48770
Before pool
After Pool
Iteration 5 of 48770
Before pool
After Pool
Iteration 6 of 48770
Before pool
After Pool
Iteration 7 of 48770
Before pool
After Pool
Iteration 8 of 48770
Before pool
After Pool
Iteration 9 of 48770
Before pool
After Pool
Iteration 10 of 48770
Before pool
After Pool
Iteration 11 of 48770
Before pool
After Pool
Iteration 12 of 48770
Before pool
After Pool
Iteration 13 of 48770
Before pool
After Pool
Iteration 14 of 48770
Before pool
After Pool
Iteration 15 of 48770
Before pool
After Pool
Iteration 16 of 48770
Before pool
After Pool
Iteration 17 of 48770
Before pool
After Pool
Iteration 18 of 48770
Before pool
After Pool
Iteration 19 of 48770
Before pool
After Pool
Iteration 20 of 48770
Before pool
After Pool
Iteration 21 of 48770
Before pool
After Pool
Iteration 22 of 4877

,Behavior,Connection,r,p,perm_p
1,cov_total,Left Amygdala_x_7Networks_LH_Vis_1,0.055048,0.879946,NaN
2,cov_total,Left Amygdala_x_7Networks_LH_Vis_1,0.055048,0.879946,NaN
3,cov_total,Left Amygdala_x_7Networks_LH_Vis_1,0.055048,0.879946,NaN
4,cov_total,Left Amygdala_x_7Networks_LH_Vis_2,-0.055048,0.879946,NaN
5,cov_total,Left Amygdala_x_7Networks_LH_Vis_1,0.055048,0.879946,NaN
...,...,...,...,...,...
1539,DASS_anx,Right Amygdala_x_Right Thalamus,0.610498,0.060845,0.9538
1540,DASS_anx,Right Amygdala_x_Right Caudate,0.283221,0.427805,0.9726
1541,DASS_anx,Right Amygdala_x_Right Putamen,0.346159,0.327176,0.9690
1542,DASS_anx,Right Amygdala_x_Right Pallidum,-0.346159,0.327176,0.9680


In [179]:
# Save dictionary using pickle
pkl_file = open("Movie_wise_DASS_anx_without_TRs.pkl", "wb")
pickle.dump(statistical_tests, pkl_file)
pkl_file.close()
statistical_tests.to_csv('Movie_wise_DASS_anx_without_TRs.csv', index=False)